In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import os
import cv2
import sys
import math
from pathlib import Path
PIPELINE_ROOT = Path('./').resolve().parent.parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from src.library.utilities.utilities_process import read_image, write_image

In [ ]:
def show_image(img, increaser=1, _title="", fsize=[15,15]):
    plt.figure(figsize=[15,15])
    plt.imshow(img * increaser, cmap="gray")
    plt.title(_title,fontsize=20)
    plt.tick_params(axis='x', labelsize=20)
    plt.tick_params(axis='y', labelsize=20)
    plt.show()

def collect_positives(uimg):
    img=np.copy(uimg)
    positives=[]
    radius = 80
    a=int(radius/2)
    color = (65000)
    min_row,min_col = startr, startc
    for index,r in df.iterrows():
        col = int(round(r['x'])-min_col)
        row = int(round(r['y'])-min_row)
        if (col<0) or (col>tile_width) or (row<0) or (row>tile_height):
            continue
        img=cv2.circle(img, (int(col), int(row)), radius, color, 20)
        I=uimg[row-a:row+a,col-a:col+a]
        if I.shape==(radius,radius):
            positives.append((col,row,I))

    return img,positives

In [ ]:
animal = 'DK184'
section = 204
channel = 3
prep_path = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'

In [ ]:
tif = str(section).zfill(3) + ".tif"
filepath = os.path.join(prep_path, f'C{channel}/full_aligned', tif)
full_img = read_image(filepath)
full_img.shape

In [ ]:
resize = False
if resize:
    filepath = os.path.join(prep_path, f'C{channel}/full_aligned', tif)
    startr = 6000
    endr = 25000
    startc = 35000
    endc = 65000
    img = full_img[startr:endr, startc:endc]
    print(img.shape)
    write_image(filepath, img)

In [ ]:
dfpath = os.path.join(prep_path, 'cell_labels', 'all_predictions.csv')
df = pd.read_csv(dfpath)
df = df.loc[df['section'] == section]
df = df.sort_values(['section', 'x', 'y'])

In [ ]:
print(df.head(3))
print(df.tail(3))

In [ ]:
count = len(df.index)
if count > 100:
    lower = count // 4
    upper = lower + 100
    print(lower, upper)
    df = df[lower:upper]
print(len(df.index))

In [ ]:
startc = int(round(min(df['x']), -3))
startr = int(round(min(df['y']), -3))
endc = math.ceil(max(df['x']))
endr = math.ceil(max(df['y']))
# manually set to ROI
#startr = 0
#endr = startr + 15000
#startc = 0
#endc = startc + 15000
print(startr, endr, startc, endc)

In [ ]:
tile_width = endc - startc
tile_height = endr - startr
img = full_img[startr:endr, startc:endc]
print(f'{startr}:{endr}, {startc}:{endc}')
print(img.shape)
#img.shape # (6800, 30000)

In [ ]:
annotated_img, positives = collect_positives(img)
print(len(positives))

In [ ]:
#vimg=annotated_img[in_row_min:in_row_max, in_col_min: in_col_max]
increaser = 15
show_image(annotated_img, increaser=increaser,_title=f'{animal} C{channel}, section: {section} predictions: {len(positives)}')

In [ ]:
plt.figure(figsize=[15,15])
end = min(100, len(positives))
for i in range(len(positives[0:end])):
    plt.subplot(10,10,i+1)
    plt.imshow(positives[i][2], cmap="gray")
plt.show()

In [ ]:
S=np.stack([I for (x,y,I)in positives])
S.shape

Mean=np.mean(S,axis=0)
Mean.shape

plt.imshow(Mean, cmap="gray")
plt.show()

_mean=np.mean(Mean.flatten())
_std=np.std(Mean.flatten())
print('before: mean=%5.2f std=%5.2f'%(_mean,_std))

Mean = (Mean-_mean)/_std

_mean=np.mean(Mean.flatten())
_std=np.std(Mean.flatten())
print('after: mean=%5.2f std=%5.2f'%(_mean,_std))

import pickle as pk
with open('mean.pkl','wb') as pkl_file:
    pk.dump({'Mean':Mean},pkl_file)

In [ ]:
new_mean = cv2.normalize(Mean, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8U)
plt.imshow(new_mean, cmap="seismic")
plt.show()

In [ ]:
l1 = ['area', 'height', 'width', 'corr_CH1', 'energy_CH1', 'corr_CH3', 'energy_CH3', 'm00CH_3', 'm10CH_3', 'm01CH_3', 'm20CH_3', 'm11CH_3', 'm02CH_3', 'm30CH_3', 'm21CH_3', 'm12CH_3', 'm03CH_3', 'mu20CH_3', 'mu11CH_3', 'mu02CH_3', 'mu30CH_3', 'mu21CH_3', 'mu12CH_3', 'mu03CH_3', 'nu20CH_3', 'nu11CH_3', 'nu02CH_3', 'nu30CH_3', 'nu21CH_3', 'nu12CH_3', 'nu03CH_3', 'h0_CH_3', 'h1_CH_3', 'h2_CH_3', 'h3_CH_3', 'h4_CH_3', 'h5_CH_3', 'h6_CH_3', 'contrast1', 'contrast3'] ['area', 'height', 'width', 'corr_CH1', 'energy_CH1', 'corr_CH3', 'energy_CH3', 'm00_mask', 'm10_mask', 'm01_mask', 'm20_mask', 'm11_mask', 'm02_mask', 'm30_mask', 'm21_mask', 'm12_mask', 'm03_mask', 'mu20_mask', 'mu11_mask', 'mu02_mask', 'mu30_mask', 'mu21_mask', 'mu12_mask', 'mu03_mask', 'nu20_mask', 'nu11_mask', 'nu02_mask', 'nu30_mask', 'nu21_mask', 'nu12_mask', 'nu03_mask', 'h0_mask', 'h1_mask', 'h2_mask', 'h3_mask', 'h4_mask', 'h5_mask', 'h6_mask', 'contrast1', 'contrast3']
l2 = [m02CH_3, h2_CH_3, nu11CH_3, nu12CH_3, m21CH_3, nu02CH_3, m10CH_3, mu11CH_3, mu21CH_3, h1_CH_3, m12CH_3, m03CH_3, mu20CH_3, mu02CH_3, nu21CH_3, nu20CH_3, nu30CH_3, h6_CH_3, nu03CH_3, h0_CH_3, h5_CH_3, h3_CH_3, mu30CH_3, mu03CH_3, h4_CH_3, m20CH_3, m30CH_3, m01CH_3, m11CH_3, m00CH_3, mu12CH_3 in input data
training data did not have the following fields: m21_mask, mu30_mask, nu30_mask, nu12_mask, mu12_mask, m01_mask, nu11_mask, nu02_mask, h1_mask, m02_mask, h4_mask, mu20_mask, h0_mask, m30_mask, m12_mask, nu21_mask, nu03_mask, m10_mask, h6_mask, mu21_mask, h3_mask, h5_mask, m11_mask, mu02_mask, m03_mask, m00_mask, mu11_mask, h2_mask, mu03_mask, m20_mask, nu20_mask